<a href="https://colab.research.google.com/github/idan424/Comp.Learning_Neuro/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter("ignore")

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
!git clone https://github.com/idan424/Comp.Learning_Neuro


Cloning into 'Comp.Learning_Neuro'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 2), reused 8 (delta 1), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [ ]:
PATH = '/content/Comp.Learning_Neuro/'
_files = ['Run_200 02.trc', 'Run_300 02.trc', 'Run_400 02.trc', 'Run_500 02.trc']
f = _files[0]

def load_data(f):
    _data = pd.read_table(PATH+f, header=4, index_col=0).iloc[:,:-1]
    _data = _data.rename(columns={'Unnamed: 1':'Time'})
    _data.reset_index(inplace=True)
    labels = pd.DataFrame(np.ones(len(_data)-2)*int(f[4:7]))
    return _data, labels

def add_norm(_data):

    for n in range(1,len(_data.columns)//3+1):
        R_data = np.linalg.norm(_data[[f"X{n}",f"Y{n}",f"Z{n}"]].values,axis=1)
        _data.insert(n*4+1,f'norm{n}',R_data)

    return _data

def add_av(_data):
    v_data = _data.copy().drop('Time', axis=1).diff()
    a_data = v_data.diff()

    v_data = v_data.rename(columns={n:f"V_{n}" for n in _data.columns})
    a_data = a_data.rename(columns={n:f"A_{n}" for n in _data.columns})

    return _data.join([v_data, a_data]).dropna()

def divide_dataset(_data, label):
    N = len(_data)//10
    return [ _data.loc[10*n:10*(n+1)-1] for n in range(N)], label[0:N]

def extract_features(seg_list):
    data = pd.DataFrame([segment_processing(seg) for seg in seg_list]) 
    return data

def segment_processing(_seg): #TODO: process a segment and concat results to the series
    seg = _seg.drop(['Time','index'], axis=1)

    mean_data = seg.mean().rename({n:f'mean_{n}'for n in seg.columns})
    std_data = seg.std().rename({n:f'std_{n}'for n in seg.columns})
    max_data = seg.max().rename({n:f'max_{n}'for n in seg.columns})
    min_data = seg.min().rename({n:f'min_{n}'for n in seg.columns})
    median_data = seg.median().rename({n:f'median_{n}'for n in seg.columns})

    # פה צריך לחשוב על עוד פיצ'רים שאנחנו רוצים להכניס לשם

    return mean_data.append([std_data,max_data,min_data,median_data])  # returns a pd.Series()



def process_file(f):
    X,y = load_data(f)
    X = add_norm(X)
    X = add_av(X)
    segments, y = divide_dataset(X, y)
    X = extract_features(segments)
    return (X,y)  # returns a tuple


files = _files
Xy = [process_file(f) for f in files]
Xy = list(zip(*Xy))  # this flip the tuple axis order

X, y = pd.concat(Xy[0]), pd.concat(Xy[1])

print(X,y)

        mean_X1      mean_Y1  ...  median_A_Z42  median_A_norm42
0   1082.295521  1095.245126  ...      0.020367        -0.778810
1   1084.604082  1064.520578  ...     -0.093186        -0.483643
2   1070.161569  1020.873275  ...      0.114094         0.818633
3   1064.539822  1058.189897  ...     -0.012860         0.285994
4   1071.028948  1100.267987  ...     -0.159018        -0.601355
..          ...          ...  ...           ...              ...
34   993.593984  1049.887616  ...     -0.043008        -0.915213
35   973.351992  1018.116769  ...      0.494933        -0.169671
36   969.060577  1000.450789  ...      0.372260         1.072537
37  1003.849861  1023.164068  ...     -0.137495        -0.500993
38  1008.346280   995.546229  ...     -0.271480        -0.081714

[190 rows x 2530 columns]         0
0   200.0
1   200.0
2   200.0
3   200.0
4   200.0
..    ...
34  500.0
35  500.0
36  500.0
37  500.0
38  500.0

[190 rows x 1 columns]


> # train-test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)


,mean_X1,mean_Y1,mean_Z1,mean_norm1,mean_X2,mean_Y2,mean_Z2,mean_norm2,mean_X3,mean_Y3,mean_Z3,mean_norm3,mean_X4,mean_Y4,mean_Z4,mean_norm4,mean_X5,mean_Y5,mean_Z5,mean_norm5,mean_X6,mean_Y6,mean_Z6,mean_norm6,mean_X7,mean_Y7,mean_Z7,mean_norm7,mean_X8,mean_Y8,mean_Z8,mean_norm8,mean_X9,mean_Y9,mean_Z9,mean_norm9,mean_X10,mean_Y10,mean_Z10,mean_norm10,...,median_A_X33,median_A_Y33,median_A_Z33,median_A_norm33,median_A_X34,median_A_Y34,median_A_Z34,median_A_norm34,median_A_X35,median_A_Y35,median_A_Z35,median_A_norm35,median_A_X36,median_A_Y36,median_A_Z36,median_A_norm36,median_A_X37,median_A_Y37,median_A_Z37,median_A_norm37,median_A_X38,median_A_Y38,median_A_Z38,median_A_norm38,median_A_X39,median_A_Y39,median_A_Z39,median_A_norm39,median_A_X40,median_A_Y40,median_A_Z40,median_A_norm40,median_A_X41,median_A_Y41,median_A_Z41,median_A_norm41,median_A_X42,median_A_Y42,median_A_Z42,median_A_norm42
46,989.511399,1071.926124,-216.113664,1474.780759,853.803140,1133.507359,-413.495702,1478.109653,837.530280,1132.464819,-299.860644,1440.094681,926.801733,962.963151,-163.324436,1346.506197,942.941824,811.676345,-190.230303,1258.670594,1052.607644,696.537373,-192.967324,1276.940195,929.138700,931.498445,-517.464422,1413.797260,805.763923,810.261360,-475.308484,1237.802862,813.335227,668.225888,-465.494861,1151.354857,1091.393979,457.353419,-253.104178,1210.130042,...,-0.285778,-0.566282,0.097560,-0.613364,-0.323578,-0.488223,0.014325,-0.547854,-0.269286,-0.441388,-0.010084,-0.530462,1.132270,-1.245308,0.240605,-0.358217,0.990533,-1.272001,-0.024862,-0.326304,1.185405,-1.279636,0.024123,-0.233600,-0.481766,-0.674805,-0.040414,-0.762580,-0.310897,-1.355128,-0.239551,-0.984078,-0.656369,-1.333693,-0.185784,-1.078064,-0.185039,-0.520686,0.186261,-0.541178
9,1003.857005,1046.763347,-223.778377,1467.501836,854.232849,1098.166839,-313.689698,1426.229898,949.624381,943.086476,-162.656019,1348.210825,983.281723,791.908599,-197.338534,1277.856309,1106.833479,695.019104,-207.847740,1323.398810,945.634211,887.594383,-519.402101,1397.142682,806.395276,789.185415,-479.423286,1226.168536,789.446927,646.198543,-467.129273,1122.477246,1234.268219,479.941046,-282.801137,1354.269821,1160.447210,426.430181,-216.465814,1255.718867,...,-1.339765,-0.853902,0.332818,-1.251345,1.887569,-2.240244,1.391063,-1.024437,1.999750,-1.716454,0.419545,-0.215711,2.252901,-1.769552,0.885470,-0.190154,-1.451505,-1.589428,-0.052032,-1.972086,0.024293,-3.586098,-0.282400,-2.695392,-0.332407,-0.838636,-0.199037,-0.841278,1.200919,-2.329378,1.384190,-1.683328,-0.761181,-3.758602,-0.598438,-2.940148,-0.377310,-0.917280,-0.123481,-0.756896
27,845.341231,1079.712215,-435.421665,1438.756783,826.339310,1079.379778,-323.101866,1397.258446,911.546840,757.565524,-217.594231,1205.100621,1009.485769,629.740384,-224.717827,1210.916286,952.695407,880.763884,-528.946222,1401.174798,868.244540,736.439838,-494.980364,1241.961790,914.914621,602.185999,-495.841255,1203.005648,1101.706591,394.550585,-305.950620,1209.864545,1020.632492,348.735009,-241.300498,1105.411135,823.758143,429.332511,-443.358152,1030.215248,...,-0.981997,-0.071494,0.090697,-0.407037,-1.079531,-0.757937,-0.478450,-0.981158,-1.323874,-0.350454,-0.275075,-0.867970,0.298784,0.619086,-0.382464,-0.162636,-1.974233,0.414172,-0.082795,-1.738741,2.132231,-1.576787,0.267194,0.498631,1.243980,-0.827895,0.251809,0.012779,0.918145,-1.158634,-0.128616,-0.089989,0.101532,1.074377,-0.190734,0.173823,-0.233221,0.715293,-0.292293,-0.171537
8,1027.412254,1044.213275,-235.422264,1483.767419,848.563994,1112.510992,-403.195798,1456.183908,860.994189,1105.563414,-290.235243,1431.085129,943.710610,925.709817,-185.033951,1334.975987,860.687105,791.947620,-202.193192,1187.367673,867.872534,636.757387,-219.246077,1099.291593,963.009693,932.410351,-536.911652,1444.039950,945.548180,767.699456,-483.244125,1310.415972,1039.721597,658.015245,-481.773373,1321.511017,743.299057,424.705362,-285.536476,903.099239,...,0.499641,-1.353636,-0.558952,-0.638398

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
_ = knn.fit(X_train, y_train)
y_predicted = knn.predict(X_test)
y_predicted = pd.DataFrame(data=y_predicted,index = y_test.index)

misclassification_rate = np.mean(y_predicted != y_test) * 100
misclassification_rate

0    0.0
dtype: float64

## _##########################################################_

In [ ]:
# X.plot()
# ax = plt.gca()
# ax.legend().remove()
# ax.figure.set_size_inches([16,200])
y_train

,0
46,300.0
9,500.0
27,400.0
8,300.0
24,400.0
...,...
23,300.0
23,400.0
45,200.0
21,300.0
